In [21]:
import pandas as pd

# Load your CSV
df = pd.read_csv('Karnataka_Datasets/Across/Sample/Karnataka_Chunk_10_90.csv')

# Convert 'Date' to datetime (format: DD-MM-YYYY)
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')

# Drop duplicates to avoid extra time steps
df = df.drop_duplicates(subset=['Latitude', 'Longitude', 'Date'])

# Sort by Latitude, Longitude, and Date
df = df.sort_values(by=['Latitude', 'Longitude', 'Date'])

# Set multi-index for easy grouping
df.set_index(['Latitude', 'Longitude', 'Date'], inplace=True)

# Columns of interest for vegetation indices and bands
columns_of_interest = ['NDVI', 'B11', 'B12', 'B3', 'B4', 'B8', 'B8A', 'GCVI']

# Bring 'Date' back as a column
df = df.reset_index()

# Recalculate TimeIndex using date ranking per point
df['TimeIndex'] = df.groupby(['Latitude', 'Longitude'])['Date'].rank(method='first').astype(int)

# Melt the dataframe to long format for pivoting
df_melt = df.melt(id_vars=['Latitude', 'Longitude', 'TimeIndex'], 
                  value_vars=columns_of_interest, 
                  var_name='Variable', 
                  value_name='Value')

# Create a new DataFrame that represents every possible (Lat, Lon, TimeIndex) combination
max_time_index = df['TimeIndex'].max()
time_index_range = pd.DataFrame({'TimeIndex': range(1, max_time_index + 1)})

# Create a grid of every Latitude, Longitude and TimeIndex combination
lat_lon_combinations = df[['Latitude', 'Longitude']].drop_duplicates()
full_grid = pd.merge(lat_lon_combinations, time_index_range, how='cross')

# Merge the original melted DataFrame with the full grid to ensure all time indices
df_full = pd.merge(full_grid, df_melt, on=['Latitude', 'Longitude', 'TimeIndex'], how='left')

# Pivot to get variables as columns (VI/Band_timeIndex)
df_pivot = df_full.pivot(index=['Latitude', 'Longitude'], 
                         columns=['Variable', 'TimeIndex'], 
                         values='Value')

# Reindex columns to ensure all (Variable, TimeIndex) combos exist
full_column_index = pd.MultiIndex.from_product(
    [columns_of_interest, range(1, max_time_index + 1)],
    names=['Variable', 'TimeIndex']
)
df_pivot = df_pivot.reindex(columns=full_column_index)

# Flatten the multi-level columns to something more readable
df_pivot.columns = [f'{var}_{i}' for var, i in df_pivot.columns]

# Reset the index for final output
df_pivot = df_pivot.reset_index()

# Merge Crop_Name back to the dataset
crop_name_df = df[['Latitude', 'Longitude', 'Crop_Name']].drop_duplicates()
df_pivot = df_pivot.merge(crop_name_df, on=['Latitude', 'Longitude'], how='left')

# Save the final result
df_pivot.to_csv('Karnataka_Datasets/', index=False)

# Optional: Print counts of available time steps per (Latitude, Longitude) pair
point_counts = df.groupby(['Latitude', 'Longitude']).size().reset_index(name='TimeSteps')
print("\n🔢 Time steps per point (Top 10):")
print(point_counts.head(10))

# Optional: Check max time steps
print(f"\n📈 Max time steps for any point: {point_counts['TimeSteps'].max()}")



🔢 Time steps per point (Top 10):
    Latitude  Longitude  TimeSteps
0  13.086502  77.475513         70
1  13.086503  77.475486         70
2  13.086532  77.473382         70
3  13.086533  77.473390         70
4  13.086534  77.473396         70
5  13.086914  77.472185         70
6  13.086940  77.472175         70
7  13.086987  77.475678         70
8  13.086998  77.472010         70
9  13.087005  77.471977         70

📈 Max time steps for any point: 70


In [2]:
import pandas as pd
import os
from glob import glob

def process_file(filepath):
    df = pd.read_csv(filepath)

    # Parse date
    df['Date'] = pd.to_datetime(df['Date'], format='mixed', dayfirst=True, errors='coerce')

    # Drop duplicates
    df = df.drop_duplicates(subset=['Latitude', 'Longitude', 'Date'])

    # Sort and reset index
    df = df.sort_values(by=['Latitude', 'Longitude', 'Date'])
    df = df.reset_index(drop=True)

    # Assign TimeIndex per (Latitude, Longitude) using date rank
    df['TimeIndex'] = df.groupby(['Latitude', 'Longitude'])['Date'].rank(method='first').astype(int)

    # Columns to use
    columns_of_interest = ['NDVI', 'B11', 'B12', 'B3', 'B4', 'B8', 'B8A', 'GCVI']

    # Melt to long format
    df_melt = df.melt(id_vars=['Latitude', 'Longitude', 'TimeIndex'], 
                      value_vars=columns_of_interest, 
                      var_name='Variable', 
                      value_name='Value')

    # Prepare full grid to ensure all time indices are retained
    max_time_index = df['TimeIndex'].max()
    time_index_range = pd.DataFrame({'TimeIndex': range(1, max_time_index + 1)})
    lat_lon_combinations = df[['Latitude', 'Longitude']].drop_duplicates()
    full_grid = pd.merge(lat_lon_combinations, time_index_range, how='cross')

    df_full = pd.merge(full_grid, df_melt, on=['Latitude', 'Longitude', 'TimeIndex'], how='left')

    # Pivot: one column per variable per time step
    df_pivot = df_full.pivot(index=['Latitude', 'Longitude'], 
                             columns=['Variable', 'TimeIndex'], 
                             values='Value')

    # Reindex columns for missing combinations
    full_column_index = pd.MultiIndex.from_product(
        [columns_of_interest, range(1, max_time_index + 1)],
        names=['Variable', 'TimeIndex']
    )
    df_pivot = df_pivot.reindex(columns=full_column_index)

    # Flatten column names
    df_pivot.columns = [f'{var}_{i}' for var, i in df_pivot.columns]
    df_pivot = df_pivot.reset_index()

    # Merge crop name back
    crop_name_df = df[['Latitude', 'Longitude', 'Crop_Name']].drop_duplicates()
    df_pivot = df_pivot.merge(crop_name_df, on=['Latitude', 'Longitude'], how='left')

    return df_pivot


# === Main logic ===

input_dir = 'Karnataka_Datasets/Across_QA/90/'  # change this as needed
output_path = os.path.join(input_dir, 'Karnataka_Merged_S2.csv')

# List all CSV files
csv_files = glob(os.path.join(input_dir, '*.csv'))

# List to hold all processed DataFrames
processed_list = []

# Process each CSV
for file in csv_files:
    print(f'📂 Processing: {os.path.basename(file)}')
    processed_df = process_file(file)
    processed_list.append(processed_df)

# Merge all processed files
final_df = pd.concat(processed_list, ignore_index=True)

# Save final merged result
final_df.to_csv(output_path, index=False)
print(f'\n✅ All files processed and saved to: {output_path}')


📂 Processing: Karnataka_Chunk_10_QA.csv
📂 Processing: Karnataka_Chunk_1_QA.csv
📂 Processing: Karnataka_Chunk_2_QA.csv
📂 Processing: Karnataka_Chunk_3_QA.csv
📂 Processing: Karnataka_Chunk_4_QA.csv
📂 Processing: Karnataka_Chunk_5_QA.csv
📂 Processing: Karnataka_Chunk_6_QA.csv
📂 Processing: Karnataka_Chunk_7_QA.csv
📂 Processing: Karnataka_Chunk_8_QA.csv
📂 Processing: Karnataka_Chunk_9_QA.csv

✅ All files processed and saved to: Karnataka_Datasets/Across_QA/90/Karnataka_Merged_S2.csv
